<a href="https://colab.research.google.com/github/AI01010/FoV_ML_spatial_audio_prediction/blob/main/FoV_ML_spatial_audio_prediction_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch torchvision joblib pandas tqdm tabulate --quiet

import time, os, torch, torch.nn as nn, torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms
from joblib import Parallel, delayed
import numpy as np, pandas as pd
from tabulate import tabulate


device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

